In [ ]:
!pip install --upgrade "huggingface_hub[cli]"

In [ ]:
from huggingface_hub import login

login()  # paste your HF to|

In [ ]:
#Load huggingface
# installations

!pip install -U fiftyone
!pip install huggingface_hub[hf_xet]
!pip install --upgrade huggingface_hub

In [ ]:
#load voxel 51 SLAKE
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub


  # Load the dataset

  # Note: other available arguments include 'max_samples', etc

dataset = load_from_hub("Voxel51/SLAKE")

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Load LLaMA
llm_model_name = "meta-llama/Llama-3.1-8B-Instruct"  # or your variant
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    dtype=torch.bfloat16  # optional, if GPU supports
)
llm_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llm_model.to(device)

In [ ]:
# --- Callable function
def run_llama(messages, max_new_tokens=300):
    inputs = llm_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(device)

    outputs = llm_model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=llm_tokenizer.eos_token_id)

    return llm_tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True
    )

In [ ]:
!pip install open_clip_torch==2.23.0 transformers==4.35.2 matplotlib

In [ ]:
from open_clip import create_model_from_pretrained, get_tokenizer # works on open-clip-torch>=2.23.0, timm>=0.9.8

model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

In [ ]:
import torch
from open_clip import create_model_and_transforms, get_tokenizer
from PIL import Image

class BiomedCLIPConfirm:
    def __init__(self,
                 model_name="hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224",
                 device=None,
                 single_rationale_threshold=20.0):
        """
        Initialize BiomedCLIP confirm module.

        Args:
            single_rationale_threshold: float, threshold for accepting a single rationale
        """
        self.device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

        # Load model, tokenizer, and preprocessing transforms
        self.model, _, self.preprocess = create_model_and_transforms(model_name)
        self.tokenizer = get_tokenizer(model_name)

        self.model.to(self.device)
        self.model.eval()
        self.context_length = 256  # Max tokens for rationales
        self.single_rationale_threshold = single_rationale_threshold

    def score_rationales(self, image, rationales, template="This is a generated rationale for an image:"):
        """
        Compute similarity scores between image and list of rationales.
        Returns a tensor of scores.
        """
        # Preprocess image
        if not isinstance(image, torch.Tensor):
            image = self.preprocess(image).unsqueeze(0)  # add batch dim
        image = image.to(self.device)

        # Tokenize rationales
        texts = self.tokenizer([template + r for r in rationales],
                               context_length=self.context_length).to(self.device)

        # Compute embeddings
        with torch.no_grad():
            image_features, text_features, logit_scale = self.model(image, texts)
            scores = (image_features @ text_features.T).squeeze(0) * logit_scale

        return scores

    def confirm(self, image, rationales, threshold_method='percentile',
                top_pct=0.3, epsilon=0.05, manual_thresh=None):
        """
        Accept/reject rationales based on similarity thresholds.

        threshold_method: 'percentile', 'relative_max', or 'manual'
        manual_thresh: float, only used if threshold_method='manual'
        """
        scores = self.score_rationales(image, rationales)

        if threshold_method == 'percentile':
            thresh = torch.quantile(scores, 1 - top_pct)
        elif threshold_method == 'relative_max':
            thresh = scores.max() - epsilon
        elif threshold_method == 'manual':
            if manual_thresh is None:
                raise ValueError("Must provide manual_thresh when using threshold_method='manual'")
            thresh = manual_thresh
        else:
            raise ValueError("threshold_method must be 'percentile', 'relative_max', or 'manual'")

        accepted = [r for r, s in zip(rationales, scores) if s >= thresh]
        rejected = [r for r, s in zip(rationales, scores) if s < thresh]

        return accepted, rejected, scores, float(thresh)

In [ ]:
# Initialize
confirm_module = BiomedCLIPConfirm()

In [ ]:
def med_vctp(image_path, question, examples5, gCaption, rCaption = None):
    # Define messages
    message1 = f"""This is the medical question at hand about a specific image: {question}. Here is a global caption describing the entire image: {gCaption}.
    Here are all regional captions for the same image that describe a certain region summarized by the label next to it: {rCaption}
    Your goal is to identify what specific information from the global and regional captions provided are useful for understanding and answering the question.
    Create a summary of the specific information that is concise yet informational and relevant to the question, so that it can be answered properly later.
    The summary can be 4 sentences maximum.
    Make the summary informative enough so that someone without access to the image can use the summary to answer the question, but make it concise enough so that someone can understand it properly"""
    messages1 = [
    {"role": "system", "content": "You are a medical visual reasoning assistant. Base your reasoning on provided medical image context and stay visually grounded and medically accurate."},
    {"role": "user", "content": message1}
    ]

    # Generate concepts & information
    responseConcepts = run_llama(messages1, max_new_tokens = 400)
    #print('1' + responseConcepts)
    message2 = f"""Here is a summary of the relevant concepts and information useful to answering the question at hand: {responseConcepts}.
    Here is the question that the summary provides useful information to answer: {question}
    The answer to this question is a single word, yes or no.
    Provided below is a list of 5 different reasoning examples, consisting of an answer to the question and a rationale quickly explaining why the answer was chosen and is correct.
    {examples5}
    Use only the summary provided to generate the rationale. Do not infer or introduce any information not in the summary.
    Use the summary intended to help create the rationale and answer the question. Separate the rationale and answer by 3 hashtags, ###.
    The rationale should be 1-2 sentences maximum and concise, and the answer should be yes or no to answer the question.
    Make sure the rationale is written so that someone can use it to immediately answer the question given at hand"""
    messages2 = [
    {"role": "system", "content": "You are a medical visual reasoning assistant. Base your reasoning on provided medical image context and stay visually grounded and medically accurate."},
    {"role": "user", "content": message2}
    ]
    # Generate rationale & answer (Conclusion)
    conclusion = run_llama(messages2 , max_new_tokens = 400)
    #print('2' + conclusion)
    if '###' in conclusion:
        parts = conclusion.split('###')
        rationale = parts[0].strip()
        answer = parts[-1].strip()  # last part should be the yes/no

    else:
      rationale, answer = conclusion, 'unknown'

    refined_rationale = rationaleCheck(question, responseConcepts, image_path, rationale, examples5)
    #print('3' + refined_rationale)
    refined_answer = run_llama(messages = [
    {"role": "system", "content": "You are a medical visual reasoning assistant. Base your reasoning on provided medical image context and stay visually grounded and medically accurate."},
    {"role": "user", "content": f"""Here is a rationale to a specific question: {refined_rationale}.
    Here is the question: {question}.
    Use the rationale to give a yes/no answer to the question. Output should be one word."""}
    ])
    #print('4' + refined_answer)

    return refined_rationale, refined_answer

In [ ]:
def rationaleCheck(question, information, image_path, rationale, examples, maxIter=3, threshold_method='manual', manual_thresh=30):
    openedImage = Image.open(image_path)

    modRationale = rationale
    best_rationale = modRationale
    best_score = -1.0  # initialize as very low

    for x in range(maxIter):

        improvMsg = [
            {"role": "system", "content": "You are a medical visual reasoning assistant. Base your reasoning on provided medical image context and stay visually grounded and medically accurate."},
            {"role": "user", "content": f"""Review this rationale: {modRationale} you generated for the question: {question}.
            If it uses irrelevant or inaccurate information, remake it based strictly on the provided information base: {information}.
Try to remake the rationale based off the information provided, and the question mentioned earlier.
Make sure that you strictly utilize information directly from the information base to generate a conclusion with a concise rationale.
Again, here are the 5 different reasoning examples consisting of an answer to the question and a rationale quickly explaining why the answer was chosen and is correct:
{examples}
**Do not use any factual information from these examples; rely solely on the summary provided.**
Make sure the rationales you generate for the given question at hand that use the information base provided earlier are 1-2 sentences maximum, written so that someone can use it to immediately answer the question"""
            }
        ]

        accepted, rejected, scores, thresh = confirm_module.confirm(
            openedImage, [modRationale], threshold_method, top_pct=0.5, manual_thresh=manual_thresh
        )

        # unwrap single-value tensor into float
        if isinstance(scores, torch.Tensor):
            current_score = float(scores.item())
        else:
            current_score = float(scores)

        # Track the highest scoring rationale
        if current_score > best_score:
            best_score = current_score
            best_rationale = modRationale

        if accepted:
            return modRationale
        else:
            modRationale = run_llama(improvMsg)

    # Fallback: if no rationale passed the threshold, return the one with highest score
    return best_rationale

In [ ]:
examples5 = """
Rationale: The image shows clear bilateral pulmonary infiltrates, consistent with fluid accumulation in both lungs. This pattern strongly indicates the condition is present. ### yes
Rationale: The enlarged heart silhouette is evident, with no signs of acute pulmonary edema. Therefore, the specific cardiac abnormality is not indicated. ### no
Rationale: The lesion appears well-circumscribed and lacks surrounding tissue invasion, suggesting it is benign. ### no
Rationale: There is a visible opacity in the lower lobe with associated pleural effusion, which supports the diagnosis. ### yes
Rationale: The imaging shows normal organ contours and no abnormal densities; there are no indicators for the condition in question. ### no"""

In [ ]:
import string

def normalize(ans):
    return ans.strip().lower().strip(string.punctuation)

In [ ]:
normal = normalize("YEs.   ")
print(normal)

In [ ]:
import json

# Load JSON files
with open("close_bbox.json") as f:
    close_bbox = json.load(f)

with open("raw_images.json") as f:
    raw_images = json.load(f)

with open("gCaptions_1.json") as f:
    gCaptions = json.load(f)

with open("rCaptions_1.json") as f:
    rCaptions = json.load(f)

# Create dictionaries for faster lookup
image_id_to_path = {item['image_id']: item['image_path'] for item in raw_images}
image_id_to_gcaption = {item['image_id']: item['caption'] for item in gCaptions}

# Organize regional captions by image_id
from collections import defaultdict
image_id_to_rcaptions = defaultdict(list)
for item in rCaptions:
    image_id_to_rcaptions[item['image_id']].append(item)

# Loop through close_bbox
correct = 0
total = 0
for entry in close_bbox:
    image_id = entry['image_id']
    question = entry['question']
    answer = entry.get('answer', None)  # optional

    # Get image path
    image_path = image_id_to_path.get(image_id, None)

    # Get global caption
    gCaption = image_id_to_gcaption.get(image_id, "")

    # Get all regional captions
    rCaptions_list = image_id_to_rcaptions.get(image_id, [])

    # Combine regional captions into one string
    rCaption_str = ""
    for rc in rCaptions_list:
        rCaption_str += f"Label: {rc['label']}, Caption: {rc['caption'].strip()}. "

    # Strip final whitespace
    rCaption_str = rCaption_str.strip()

    # Now you have all components for the VCTP function
    rationaleOut, answerOut = med_vctp(image_path, question, examples5, gCaption, rCaption_str)
    if normalize(answerOut) == answer.lower():
        correct+=1
    else:
        pass
    total+=1
    if total%100 == 0:
        print(f"100 done so far, {correct} correct and {total} total questions passed")
print(f"Total correct = {correct}")
print(f"Total = {total}")

In [ ]:
print(correct)

In [ ]:
print(total)

In [ ]:
def med_vctp1(image_path, question, examples5, gCaption, rCaption = None):
    # Define messages
    message1 = f"""This is the medical question at hand about a specific image: {question}. Here is a global caption describing the entire image: {gCaption}.
    Here are all regional captions for the same image that describe a certain region summarized by the label next to it: {rCaption}
    Your goal is to identify what specific information from the global and regional captions provided are useful for understanding and answering the question.
    Create a summary of the specific information that is concise yet informational and relevant to the question, so that it can be answered properly later.
    The summary can be 4 sentences maximum.
    Make the summary informative enough so that someone without access to the image can use the summary to answer the question, but make it concise enough so that someone can understand it properly"""
    messages1 = [
    {"role": "system", "content": "You are a medical visual reasoning assistant. Base your reasoning on provided medical image context and stay visually grounded and medically accurate."},
    {"role": "user", "content": message1}
    ]

    # Generate concepts & information
    responseConcepts = run_llama(messages1, max_new_tokens = 400)
    #print('1' + responseConcepts)
    message2 = f"""Here is a summary of the relevant concepts and information useful to answering the question at hand: {responseConcepts}.
    Here is the question that the summary provides useful information to answer: {question}
    The answer to this question is a single word, yes or no.
    Provided below is a list of 5 different reasoning examples, consisting of an answer to the question and a rationale quickly explaining why the answer was chosen and is correct.
    {examples5}
    Use only the summary provided to generate the rationale. Do not infer or introduce any information not in the summary.
    Use the summary intended to help create the rationale and answer the question. Separate the rationale and answer by 3 hashtags, ###.
    The rationale should be 1-2 sentences maximum and concise, and the answer should be yes or no to answer the question.
    Make sure the rationale is written so that someone can use it to immediately answer the question given at hand"""
    messages2 = [
    {"role": "system", "content": "You are a medical visual reasoning assistant. Base your reasoning on provided medical image context and stay visually grounded and medically accurate."},
    {"role": "user", "content": message2}
    ]
    # Generate rationale & answer (Conclusion)
    conclusion = run_llama(messages2 , max_new_tokens = 400)
    rationale, answer = parse_vctp_output(conclusion)
    #print('2' + conclusion)
    return rationale, answer

In [ ]:
def parse_vctp_output(conclusion):
    # Lowercase for normalization
    conclusion = conclusion.lower().strip()

    # Remove leading numbers like "2" or "2###"
    conclusion = conclusion.lstrip("0123456789. ").replace("2###", "").strip()

    # Extract yes/no
    if conclusion.startswith("yes"):
        answer = "yes"
        rationale = conclusion[len("yes"):].strip(" \n#")
    elif conclusion.startswith("no"):
        answer = "no"
        rationale = conclusion[len("no"):].strip(" \n#")
    else:
        # fallback: search for 'yes' or 'no' in the first 20 chars
        if "yes" in conclusion[:20]:
            answer = "yes"
        elif "no" in conclusion[:20]:
            answer = "no"
        else:
            answer = "unknown"
        rationale = conclusion
    return rationale, answer

In [ ]:
import json

# Load JSON files
with open("close_bbox.json") as f:
    close_bbox = json.load(f)

with open("raw_images.json") as f:
    raw_images = json.load(f)

with open("gCaptions_1.json") as f:
    gCaptions = json.load(f)

with open("rCaptions_1.json") as f:
    rCaptions = json.load(f)

# Create dictionaries for faster lookup
image_id_to_path = {item['image_id']: item['image_path'] for item in raw_images}
image_id_to_gcaption = {item['image_id']: item['caption'] for item in gCaptions}

# Organize regional captions by image_id
from collections import defaultdict
image_id_to_rcaptions = defaultdict(list)
for item in rCaptions:
    image_id_to_rcaptions[item['image_id']].append(item)

# Loop through close_bbox
correct = 0
total = 0
for entry in close_bbox:
    image_id = entry['image_id']
    question = entry['question']
    answer = entry.get('answer', None)  # optional

    # Get image path
    image_path = image_id_to_path.get(image_id, None)

    # Get global caption
    gCaption = image_id_to_gcaption.get(image_id, "")

    # Get all regional captions
    rCaptions_list = image_id_to_rcaptions.get(image_id, [])

    # Combine regional captions into one string
    rCaption_str = ""
    for rc in rCaptions_list:
        rCaption_str += f"Label: {rc['label']}, Caption: {rc['caption'].strip()}. "

    # Strip final whitespace
    rCaption_str = rCaption_str.strip()

    # Now you have all components for the VCTP function
    rationaleOut, answerOut = med_vctp1(image_path, question, examples5, gCaption, rCaption_str)
    if normalize(answerOut) == answer.lower():
        correct+=1
    else:
        pass
    total+=1
    if total%100 == 0:
        print(f"100 done so far, {correct} correct and {total} total questions passed")
print(f"Total correct = {correct}")
print(f"Total = {total}")

In [ ]:
print(correct)

In [ ]:
print(total)